In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
%load_ext autoreload
%autoreload 2
from strategy.backtest import Backtest
from strategy.data import RawDataUniV3, PoolDataUniV3
from strategy.positions import UniV3Position
from strategy.viewers import PotrfolioViewer, RebalanceViewer, UniswapViewer, LiquidityViewer
from strategy.strategies import AbstractStrategy
from strategy.uniswap_utils import UniswapLiquidityAligner
from strategy.primitives import Pool, Token, Fee

In [ ]:
def tick_to_price(tick, decimal_diff=10):
    price = np.power(1.0001, tick) / 10 ** decimal_diff
    return price

def price_to_tick(price, decimal_diff=10):
    tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
    return int(round(tick))

## Synthetic Data

In [ ]:
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)

In [ ]:
class LinearData:
    def __init__(self, pool, start_date='1-1-2022', freq='4H', n_points=2190, init_price=1, growth_rate=0.05):
        self.pool = pool
        self.start_date = start_date
        self.freq = freq
        self.n_points = n_points
        self.init_price = init_price
        self.growth_rate = growth_rate

    def generate_data(self):
        timestamps = pd.date_range(start=self.start_date, periods=self.n_points, freq=self.freq, normalize=True)
        prices = np.linspace(self.init_price, self.init_price * (1 + self.growth_rate), self.n_points)

        df = pd.DataFrame(data=zip(timestamps, prices), columns=['timestamp', 'price']).set_index('timestamp')

        df["price_before"] = df["price"].shift(1)
        df["price_before"] = df["price_before"].bfill()

        df["price_next"] = df["price"].shift(-1)
        df["price_next"] = df["price_next"].ffill()

        return PoolDataUniV3(self.pool, mints=None, burns=None, swaps=df)



In [ ]:
data = LinearData(pool).generate_data()

In [ ]:
# data = RawDataUniV3(pool).load_from_folder()

## Passive UniV3 strategy

In [ ]:
class UniV3Passive(AbstractStrategy):
    """
    ``UniV3Passive`` is the passive strategy on UniswapV3 without rebalances.
        lower_price: Lower bound of the interval
        upper_price: Upper bound of the interval
        rebalance_cost: Rebalancing cost, expressed in currency
        pool: UniswapV3 Pool instance
        name: Unique name for the instance
    """
    def __init__(self,
                 lower_price: float,
                 upper_price: float,
                 pool: Pool,
                 rebalance_cost: float,
                 name: str = None,
                 ):
        super().__init__(name)
        self.lower_price = lower_price
        self.upper_price = upper_price
        self.decimal_diff = -pool.decimals_diff
        self.fee_percent = pool.fee.percent
        self.rebalance_cost = rebalance_cost

    def rebalance(self, *args, **kwargs) -> bool:
        timestamp = kwargs['timestamp']
        row = kwargs['row']
        portfolio = kwargs['portfolio']
        price_before, price = row['price_before'], row['price']

        is_rebalanced = None

        if len(portfolio.positions) == 0:
            univ3_pos = self.create_uni_position(price)
            portfolio.append(univ3_pos)
            is_rebalanced = 'mint'

        if 'UniV3Passive' in portfolio.positions:
            uni_pos = portfolio.get_position('UniV3Passive')
            uni_pos.charge_fees(price_before, price)

        return is_rebalanced


    def create_uni_position(self, price):
        uni_aligner = UniswapLiquidityAligner(self.lower_price, self.upper_price)
        
        x_uni_aligned, y_uni_aligned = uni_aligner.align_to_liq(1 / price, 1, price)
        
        univ3_pos = UniV3Position('UniV3Passive', self.lower_price, self.upper_price, self.fee_percent, self.rebalance_cost)
        univ3_pos.deposit(x_uni_aligned, y_uni_aligned, price)

        return univ3_pos

In [ ]:
univ3_passive = UniV3Passive(data.swaps['price'].min() - 1e-3, data.swaps['price'].max() + 1e-3, pool, 0.01)

In [ ]:
b = Backtest(univ3_passive)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
uv = UniswapViewer(uni_history)
uni_interval_fig = uv.draw_intervals(data.swaps)

In [ ]:
uni_interval_fig

In [ ]:
uni_history.get_coverage(data.swaps)

In [ ]:
fig1, fig2, fig3, fig4, fig5 = PotrfolioViewer(portfolio_history, pool).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig3

In [ ]:
fig2

In [ ]:
fig4

In [ ]:
fig5